<a href="https://colab.research.google.com/github/Adityakishore0/Adityakishore0/blob/main/vidyaai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask flask_cors diffusers transformers torch torchvision torchaudio scipy numpy xformers  pyngrok

INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 8

In [ ]:
import torch
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
from diffusers import StableDiffusionPipeline
import uuid
import os
from pyngrok import ngrok

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Create a directory for storing images
IMAGE_FOLDER = "static/images"
os.makedirs(IMAGE_FOLDER, exist_ok=True)

# Load Stable Diffusion model
model_id = "runwayml/stable-diffusion-v1-5"

# Check if GPU is available, otherwise fallback to CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16 if device == "cuda" else torch.float32)
pipe.to(device)

# Optimize for low VRAM (only if using GPU)
if device == "cuda":
    pipe.enable_attention_slicing()
    pipe.enable_xformers_memory_efficient_attention()

# Start Flask server with ngrok
public_url = ngrok.connect(5000).public_url
print(f"🔗 Public API URL: {public_url}")

# Image generation route
@app.route('/generate', methods=['POST'])
def generate_image():
    try:
        data = request.json
        prompt = data.get("prompt", "")

        if not prompt:
            return jsonify({"error": "No prompt provided"}), 400

        # Generate image
        image = pipe(prompt, height=512, width=512).images[0]

        # Save image with a unique filename
        filename = f"generated_{uuid.uuid4()}.png"
        image_path = os.path.join(IMAGE_FOLDER, filename)
        image.save(image_path)

        # Generate a public URL for the image
        image_url = f"{public_url}/images/{filename}"

        return jsonify({"image": image_url})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Route to serve images
@app.route('/images/<filename>')
def serve_image(filename):
    return send_from_directory(IMAGE_FOLDER, filename)

if __name__ == "__main__":
    app.run(port=5000, host="0.0.0.0")


Using device: cpu


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

🔗 Public API URL: https://4ade-34-72-28-86.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 08:43:26] "POST /generate HTTP/1.1" 200 -


In [3]:
ngrok.set_auth_token("2tWlTbY327eFqxyUH7NqZ4qK13L_2PboAfWG1DNaswWGajwAM")
